In [95]:
# Load Libraries and Data
# download data from https://www.eia.gov/electricity/data/eia826/

library(zoo)
library(reshape2)


Attaching package: ‘reshape2’

The following objects are masked from ‘package:reshape’:

    colsplit, melt, recast



In [137]:
check4data <- function(file, reverse = FALSE) {
    numlines = 100
    s = readLines(file)
    if(reverse == FALSE) {
        checkLines = c(1:numlines)
    } else {
        tLines = length(s)
        checkLines = rev(c((tLines - numlines):tLines))
    }
    for (i in checkLines) {
        lineVec = unlist(strsplit(s[i], split=","))
        if (is.na(as.integer(lineVec[1])) == F) {
            return(i)
        }
    }
    return(0)
}
revandcollapse <- function(x) {
    x = rev(x)
    x[which(x == "")] <- NA
    x = na.locf(x)
    return(x)
}

extractheaderinfo <- function(file, lines) {
    raw = readLines(file, n = lines)
    for (i in rev(c(1:lines))) {
        lineVec = unlist(strsplit(raw[i], split=","))
        # the above misses the last element if it's blank.
        if(i != lines) {
            if(length(lineVec) < length(prevline)) {
                lineVec = append(lineVec,"", after = length(lineVec))
            }
        }
        lineVec[which(lineVec == "")] <- NA
        t = na.locf(lineVec, na.rm = FALSE)
        t[which(is.na(t))] <- ""
        if(i == lines) {
            prevline = t
        } else {
            prevline = paste(prevline, t, sep = "|")
        }
    }
    return(sapply(strsplit(as.character(prevline), "\\|"), function(x) revandcollapse(x)))
}

splitdata <- function(data, headers, catlen = 4) {
    els = sapply(headers, function(x) length(x))
    chidxs = which(els <= 1)
    uhidxs = setdiff(c(1:length(els)), chidxs)
    tdata = data[,c(chidxs, 5:8)]
    print(sapply(tdata,class))
    t = melt(tdata, id=1:4)
    # sapply(headers, function(x) x[1])[chidxs]
    # http://www.statmethods.net/management/reshape.html
    print(t)
}
                           
c2num <- function(data, headers) {
    els = sapply(headers, function(x) length(x))
    chidxs = which(els <= 1)
    uhidxs = setdiff(c(1:length(els)), chidxs)
    data[uhidxs] <- lapply(data[uhidxs], function(x) as.numeric(gsub(",", "",as.character(x))))
    data[is.na(data)] <- 0
    return(data)
}

file = "data/sales_revenue_1.csv"
firstline = check4data(file) - 1
# [1] 4
lastline = check4data(file, reverse = TRUE) - firstline
# [1] 16014

alldata = read.csv(file, header = FALSE, skip = firstline, nrows = lastline)
alldata = c2num(alldata, headers)
headers = extractheaderinfo(file,firstline)
splitdata(alldata[1,], headers)


Warning message:
: NAs introduced by coercionWarning message:
: NAs introduced by coercionWarning message:
: NAs introduced by coercionWarning message:
: NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced by coercionWarning message:
In FUN(X[[i]], ...): NAs introduced

       V1        V2        V3        V4        V5        V6        V7        V8 
"integer" "integer"  "factor"  "factor" "numeric" "numeric" "numeric" "numeric" 
    V1 V2 V3    V4 variable     value
1 1990  1 AK Final       V5  17477.00
2 1990  1 AK Final       V6 181752.00
3 1990  1 AK Final       V7      0.00
4 1990  1 AK Final       V8      9.62
